In this notebook we estimate the point values per court location, similar to this graphic https://www.instagram.com/p/DA_IjDyxtdv/?igsh=d2FmY2FtejBwbWw0 

In [1]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from euroleague_api.shot_data import ShotData

sys.path.append("../utils/")
# from shot_chart_plots import fg_perc_hex_heatmap
from draw_court import draw_court
from get_new_zones import hexagon_zones
# %matplotlib

plt.close("all")

In [3]:
def plot_heatmap(df, gridsize, szn_min, szn_max):
    x = df["COORD_X"]
    y = df["COORD_Y"]
    z = df["ratio"]

    if szn_min == szn_max:
        title = f"Season {szn_max}"
    else:
        title = f"Seasons {szn_min} - {szn_max}"

    cmap = plt.cm.PiYG
    fig, ax = plt.subplots()
    hb = ax.hexbin(x=x, y=y, C=z, gridsize=gridsize,
                    extent=[-800, 800, -200, 1300],
                    cmap=cmap, alpha=0.8, vmin=0., vmax=2)
    cb = fig.colorbar(hb)
    ax = draw_court(ax=ax, background=True, elogo=False, watermark=True)
    plt.title(title)
    plt.show()
    return


def point_value_by_hex_loc(df, gridsize=20, threshold=2.0):
    df = hexagon_zones(df.copy(), gridsize=gridsize)
    pivot_df = df.pivot_table(index="ZONE_HEX", columns="ID_ACTION", aggfunc="count")["ACTION"]
    pivot_df["IS2PT"] = np.where(pivot_df["2FGA"].fillna(0) > pivot_df["3FGA"].fillna(0), True, False)

    two_fg_df = 2 * pivot_df.loc[pivot_df["IS2PT"], "2FGM"] / pivot_df.loc[pivot_df["IS2PT"], ["2FGA", "2FGM"]].sum(axis=1)
    three_fg_df = 3 * pivot_df.loc[~pivot_df["IS2PT"], "3FGM"] / pivot_df.loc[~pivot_df["IS2PT"], ["3FGA", "3FGM"]].sum(axis=1)

    fg_df = pd.concat([two_fg_df[two_fg_df.notna()], three_fg_df[three_fg_df.notna()]])
    print("Consistency check:", fg_df.shape, fg_df.index.unique().shape)
    plot_df = fg_df.to_frame("ratio").reset_index().merge(df[["COORD_X", "COORD_Y", "ZONE_HEX"]], how="left")
    if not plot_df.groupby("ZONE_HEX")["ratio"].max().equals(plot_df.groupby("ZONE_HEX")["ratio"].min()):
        print("Something is wrong after the merge.")
    plot_df = plot_df[plot_df["ratio"] < threshold]

    szn_min = df["Season"].min()
    szn_max = df["Season"].max()
    plot_heatmap(plot_df, gridsize, szn_min=szn_min, szn_max=szn_max)
    return plot_df

In [4]:
# Read input data (extracted elsewhere)
file = "~/Documents/euroleague_api/notebooks/data/shot_data_2007_2023.csv"
shot_df = pd.read_csv(file)

In [ ]:
shot_df.shape

In [6]:
gridsize = 20

In [7]:
# snz_shot_df = corner_three_zones(snz_shot_df)
# snz_shot_df = hexagon_zones(snz_shot_df, gridsize=gridsize)
# fg_perc_hex_heatmap(snz_shot_df, gridsize=gridsize)

# Season 2023 - 2024

In [ ]:
snz_shot_df = shot_df[(shot_df["Season"] == 2023)]
_ = point_value_by_hex_loc(snz_shot_df)

# Seasons 2019 - 2023

In [ ]:
_ = snz_shot_df = shot_df[(shot_df["Season"] <= 2023) & (shot_df["Season"] >= 2019)]
point_value_by_hex_loc(snz_shot_df)

# Seasons 2007 - 2023

In [ ]:
_ = snz_shot_df = shot_df.copy()
point_value_by_hex_loc(snz_shot_df)